<span style="font-family:IBM Plex Mono">**Week 4 Homework | Analytics Engineering**</span>    
---
> In this homework, we'll use the models developed during the week 4 videos and enhance the already presented dbt project using the already loaded Taxi data for fhv vehicles for year 2019 in our DWH.

> **NOTE:** 

<span style="font-family:IBM Plex Mono">This means that in this homework we use the following data [Datasets list](https://github.com/DataTalksClub/nyc-tlc-data/)
* Yellow taxi data - Years 2019 and 2020
* Green taxi data - Years 2019 and 2020 
* fhv data - Year 2019. 

We will use the data loaded for:

* Building a source table: `stg_fhv_tripdata`
* Building a fact table: `fact_fhv_trips`
* Create a dashboard 

If you don't have access to GCP, you can do this locally using the ingested data from your Postgres database
instead. If you have access to GCP, you don't need to do it for local Postgres -
only if you want to.

> **Note**: if your answer doesn't match exactly, select the closest option 
</span>


In [1]:
# imports
import os
from datetime import datetime
from pathlib import Path
import urllib.request
import pandas as pd
from prefect import task, flow
from prefect_gcp import GcpCredentials
from google.cloud import bigquery
print("Setup Complete")

Setup Complete


### Question 1: 

**What is the count of records in the model fact_trips after running all models with the test run variable disabled and filtering for 2019 and 2020 data only (pickup datetime)?** 

You'll need to have completed the ["Build the first dbt models"](https://www.youtube.com/watch?v=UVI30Vxzd6c) video and have been able to run the models via the CLI. 
You should find the views and models for querying in your DWH.

- 41648442
- 51648442
- 61648442
- 71648442



In [7]:
# Load GCP creds
@flow()
def get_bigquery_client():
    gcp_creds_block = GcpCredentials.load("ny-taxi-gcp-creds")
    gcp_creds = gcp_creds_block.get_credentials_from_service_account()
    client = bigquery.Client(credentials=gcp_creds)
    return client

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/flows.py:206: UserWarning: A flow named 'get-bigquery-client' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_42036/1825830144.py:2' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


In [8]:
#create a client
client = get_bigquery_client()
# Construct a reference to the "ny_taxi_bucket_de_2023" dataset
dataset_ref = client.dataset(dataset_id="ny_taxi", project="dtc-de-2023")
# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)
#construct a table reference to "ny_taxi_tripdata_2019"
table_ref = dataset_ref.table("green_tripdata_2019")
#API request
table_ny_taxi_tripdata = client.get_table(table_ref)
#to dataframe
df_ny_taxi_tripdata = client.list_rows(table=table_ny_taxi_tripdata, max_results=5).to_dataframe()
df_ny_taxi_tripdata.head()

12:06:37.476 | INFO    | prefect.engine - Created flow run 'ethereal-swine' for flow 'get-bigquery-client'

12:06:40.947 | INFO    | Flow run 'ethereal-swine' - Finished in state Completed()

,VendorID,pickup_datetime,dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,category
0,1,2019-04-01 04:56:51+00:00,2019-04-01 05:07:16+00:00,N,5,60,126,1,2.7,0.0,...,0.0,0.0,0.0,NaN,0.0,0.0,2,2,0.0,green
1,1,2019-04-01 05:29:57+00:00,2019-04-01 06:00:05+00:00,N,5,208,212,1,9.5,0.0,...,0.0,0.0,0.0,NaN,0.0,0.0,2,2,0.0,green
2,2,2019-04-01 05:31:37+00:00,2019-04-01 05:31:37+00:00,N,1,193,264,1,0.0,0.0,...,0.0,0.0,0.0,NaN,0.0,0.0,2,1,0.0,green
3,1,2019-04-01 06:00:21+00:00,2019-04-01 06:17:30+00:00,N,5,212,183,1,5.3,0.0,...,0.0,0.0,0.0,NaN,0.0,0.0,2,2,0.0,green
4,1,2019-04-01 06:18:11+00:00,2019-04-01 06:23:47+00:00,N,5,183,3,1,1.5,0.0,...,0.0,0.0,0.0,NaN,0.0,0.0,2,2,0.0,green
